In [3]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 5.1 MB/s 
     |████████████████████████████████| 60 kB 4.6 MB/s 


In [16]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que es posible tener multiples agentes una sola celda elegimos `MultipleGrid`.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import math

# Librerias para mostrar tablas
import requests
import io

In [5]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, cleanerAgent):
                grid[x][y] = 2
            elif grid[x][y] == 0:
                grid[x][y] = content.dirty
    return grid

class RandomWalker(Agent):
    
    '''
    Referencia: https://github.com/projectmesa/mesa/blob/main/examples/wolf_sheep/wolf_sheep/random_walk.py
    '''
    


    """
    Class implementing random walker methods in a generalized manner.
    Not indended to be used on its own, but to inherit its methods to multiple
    other agents.
    """

    grid = None
    x = None
    y = None

    def __init__(self, unique_id, pos, model):
        """
        grid: The MultiGrid object in which the agent lives.
        x: The agent's current x coordinate
        y: The agent's current y coordinate
        """
        super().__init__(unique_id, model)
        self.pos = pos

    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, True, False)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

class cleanerAgent(RandomWalker):
    # Representa a un agente limpiador que se encarga de limipar las celdas sucias

    def __init__(self, unique_id, pos, model):
      # Crea un agente de cleanerAgent

      super().__init__(unique_id, pos, model)

    def step(self):
        '''
        Este metodo se encarga de verificar si la celda en la que se encuentra el agente
        esta sucio o limpio y en el caso de que este sucio lo limpia y si no se seguira movimiendo
        '''
        cleaning = False
        cell = self.model.grid.get_cell_list_contents([self.pos])
        for content in cell:
            if isinstance(content, floorAgent):
                if content.dirty:
                    content.dirty = 0
                    cleaning = True
        if not cleaning:
            self.random_move()
            

class floorAgent(Agent):
    
    # Representa a un agente o celda de estado sucio(1) o limpio(0)

    def __init__(self, unique_id, model, pos, dirty):

        # Crea un agente con estado inicial de sucio o limpio

        super().__init__(unique_id, model)
        self.pos = pos
        self.dirty = dirty
                
            
class cleanerRoomModel(Model):

    # Define el modelo de la actividad.

    def __init__(self, width, height, numCleaners, numDirtyTiles):
        super().__init__()
        self.height = height
        self.width = width
        self.grid = MultiGrid(self.width, self.height, True)
        self.schedule = SimultaneousActivation(self)

        self.numCleaners = numCleaners
        self.numDirtyTiles = numDirtyTiles

        self.numStepsDone = 0;
        self.numTilesCleaned = 0;
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
        # Crear agentes limpiadores:
        for i in range(self.numCleaners):
            tempCleaner = cleanerAgent(self.next_id(), (1, 1), self)
            self.grid.place_agent(tempCleaner, (1, 1))
            self.schedule.add(tempCleaner)
            
        # Crear tiles sucias y limpias
        countDirtyTiles = 0
        for agent, x, y in self.grid.coord_iter():
            if countDirtyTiles >= self.numDirtyTiles:
              dirty = 0
            else:
              dirty = self.random.choice([0, 1])

            if dirty:
                countDirtyTiles += 1

            floorTile = floorAgent(self.next_id(), self, (x, y), dirty)
            self.grid.place_agent(floorTile, (x, y))
            self.schedule.add(floorTile)
        
    def addCleanedTile(self):
      self.numTilesCleaned += 1

    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()
        self.numStepsDone += 1
        
        temp = 0

        for (null, x, y) in self.grid.coord_iter():
          cell = self.grid.get_cell_list_contents([(x,y)])
          for content in cell:
              if isinstance(content, floorAgent):
                  if content.dirty:
                      temp += 1
                      

        self.numTilesCleaned = temp
                    
        

        return [self.numTilesCleaned, self.numStepsDone]

In [6]:
# Definimos el tamaño del Grid
m = 10
n = 10

# Definimos el porcentaje de casillas sucias, el numero de aspiradoras y el tiempo maximo
# Porcentaje de casillas sucias
percentageDirtyTiles = 50 # Porciento
# Tiempo de execucion maximo
maxTime = 0.5 # En segundos
# Numero de Aspiradoras
numCleaners = 30

# Registramos el tiempo de inicio y limite
startTime = time.time()
limitTime = datetime.timedelta(seconds=maxTime)

# Variables temporales
temp = [1,0]
numDirtyTiles = int((m * n) * (percentageDirtyTiles/100))

#Inicializamos el modelo y realizamos los steps
model = cleanerRoomModel(m, n, numCleaners, numDirtyTiles)
while (datetime.timedelta(seconds=(time.time() - startTime)) < limitTime) and 0 < temp[0]:
    temp = model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - startTime))))
print('Movimientos realizados por cada Aspiradora:', temp[1])
print('Porcentaje de celdas limpias:', (n*m-temp[0])/(n*m) * 100, "%")

print(str(datetime.timedelta(seconds=(time.time() - startTime))))
print(temp[1])
print((n*m-temp[0])/(n*m) * 100)


Tiempo de ejecución: 0:00:00.040732
Movimientos realizados por cada Aspiradora: 49
Porcentaje de celdas limpias: 100.0 %
0:00:00.041220
49
100.0


In [7]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [8]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
anim

# Conclusiones
Tras concluir con la programacion del modelo se llevaron a cabo varias pruebas para determinar los efectos de cambiar la cantidad de agentes limpiadores que actuan sobre el ambiente. Al realizar las pruebas se mentubieron algunas constantes, el grid siempre fue de 10 x 10, y la cantidad de celdas sucias siempre fue el 50% de la cantidad de celdas total(100).


En la siguiente seccion se muestran los resultados:

In [19]:
# Aqui se sube accede al archivo resultados.csv que se encuentra en el repositorio de GitHub junto a este Jupyter notebook

url = "https://ghcdn.rawgit.org/facund015/MultiagentesPersonal/main/Modulo%201/Actividad_1/resultados.csv"
download = requests.get(url).content

df = pd.read_csv(io.StringIO(download.decode('utf-8')))
print(df)

    numero_agentes  porcentaje_celdas_limpias  numero_steps  tiempo_ejecucion
0                3                        100           216          0.133502
1                3                        100           194          0.106915
2                3                        100           191          0.105101
3                3                        100           272          0.148933
4                3                        100           261          0.148028
5                3                        100           152          0.088122
6                3                        100           172          0.102275
7                3                        100           202          0.118706
8                3                        100           197          0.110651
9                3                        100           214          0.129075
10              10                        100            61          0.047698
11              10                        100           142     

Como se puede observar en la tabla anterior, hay una relacion directa entre la cantidad de agentes limpiadores, y el tiempo de ejecucion, mientras vamos aumentando los agentes, mas baja el tiempo de ejecucion.

De igual manera podemos notar como el elemento del azar del sistema de agentes es mas notable cuanto menos agentes limpiadores se encuentran en el ambiente. La diferencia entre los tiempos de ejecucion en los casos con la misma cantidad de agentes limpiadores va disminuyendo mientras vamos anadiendo mas agentes.

